In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import warnings
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings("ignore")  # 忽略一些警告 不影响运行
from PyEMD import EEMD, CEEMDAN, EMD
import numpy as np

In [2]:
data = pd.read_csv('sw_index_1.csv')

df = data.drop('date', axis=1)
# print(df.shape)
# exit()


import logging
# 设置信号选项
N = len(df)
tMin, tMax = 0, 2 * np.pi
T = np.linspace(tMin, tMax, N)

# 初始化CEEMDAN对象
trials = 20
ceemdan = CEEMDAN(trials=trials)

# 准备存储分解结果的字典
decomposed_results = {}
# 对df中的每个信号进行CEEMDAN分解
for column in df.columns:
    S = df[column].values  # 获取当前列的数据
    C_IMFs = ceemdan(S, T)  # 执行CEEMDAN分解
    decomposed_results[column] = C_IMFs  # 将分解结果存储到字典中

new_df = pd.DataFrame()
# 将分解结果添加到新的DataFrame中
for i, (column, IMFs) in enumerate(decomposed_results.items()):
    for imf_no, IMF in enumerate(IMFs):
        new_column_name = f'{column}_IMF_{imf_no + 1}'
        new_df[new_column_name] = IMF

# import matplotlib.pyplot as plt
#
deco_df = pd.DataFrame()

# 遍历decomposed_results字典中的每个键（即每列的名称）
for column in decomposed_results.keys():
    # 获取当前列的所有IMF分量
    IMFs = decomposed_results[column]
    # 去除第一个IMF分量
    IMFs_without_first = IMFs[1:]
    # 叠加剩余的IMF分量以得到去噪后的数据
    denoised_signal = np.sum(IMFs_without_first, axis=0)
    # 将去噪后的信号作为新列添加到deco_df中，并以列名后缀"_denoised"作为新列名
    deco_df[column + "_denoised"] = denoised_signal
print(deco_df.shape)

(2434, 31)


In [3]:
from minepy import MINE

In [4]:
mic_df = pd.DataFrame()

# 假设我们要对deco_df中的所有列进行两两MIC相关性分析
for i in range(len(deco_df.columns)):
    for j in range(i + 1, len(deco_df.columns)):
        # 获取两个列的数据
        data1 = deco_df.iloc[:, i]
        data2 = deco_df.iloc[:, j]

        # 初始化MINE对象并计算MIC系数
        mine = MINE()
        mine.compute_score(data1, data2)

        # 将结果添加到mic_df中
        mic_df = mic_df.append({'Baseline': data1.name, 'Correlated': data2.name, 'MIC': mine.mic()},
                               ignore_index=True)

# 现在mic_df包含了所有的MIC相关性分析结果
# 我们可以对mic_df进行排序，以便查看最高和最低的MIC系数
mic_df = mic_df.sort_values(by='MIC', ascending=False)

mic_df.to_csv('mic_666.csv', index=False)
print(mic_df)

          Baseline     Correlated       MIC
48     环保_denoised  纺织服饰_denoised  0.898647
319  社会服务_denoised  食品饮料_denoised  0.897288
363  交通运输_denoised  纺织服饰_denoised  0.892675
44     环保_denoised  交通运输_denoised  0.883584
240  建筑装饰_denoised  交通运输_denoised  0.882648
..             ...            ...       ...
170  国防军工_denoised    银行_denoised  0.260991
25   美容护理_denoised    钢铁_denoised  0.255385
84   石油石化_denoised  农林牧渔_denoised  0.253094
68   石油石化_denoised    银行_denoised  0.241571
95     煤炭_denoised    银行_denoised  0.230210

[465 rows x 3 columns]


In [5]:
deco_df

,美容护理_denoised,环保_denoised,石油石化_denoised,煤炭_denoised,通信_denoised,计算机_denoised,国防军工_denoised,机械设备_denoised,电力设备_denoised,建筑装饰_denoised,...,食品饮料_denoised,家用电器_denoised,汽车_denoised,电子_denoised,有色金属_denoised,钢铁_denoised,基础化工_denoised,农林牧渔_denoised,传媒_denoised,非银金融_denoised
0,4409.405020,2319.401340,1916.860987,1639.384583,1429.357643,2433.027755,949.639970,933.971707,3355.406168,1528.767037,...,4596.634046,2920.743091,2815.986956,1768.680178,2380.227057,1545.961929,1800.211338,1884.951389,872.939147,1051.015657
1,4394.790587,2325.420011,1904.952997,1614.101041,1427.538818,2433.912423,944.355466,930.618853,3357.589561,1507.367967,...,4571.131288,2874.368710,2790.577200,1765.702394,2381.573326,1547.344783,1788.297492,1870.320546,868.993351,1044.288128
2,4358.364104,2322.143201,1891.392581,1575.822585,1422.431742,2436.182944,935.708331,924.980418,3361.627542,1480.498315,...,4560.641529,2835.416097,2755.736499,1772.939194,2360.924150,1532.396481,1773.965966,1844.962918,877.387460,1034.582192
3,4332.896301,2314.557970,1889.064359,1546.416197,1418.182561,2438.274888,925.034868,916.821174,3358.405285,1459.828815,...,4509.009765,2813.817658,2718.440633,1777.441119,2336.109135,1504.320444,1751.884244,1814.583060,890.642598,1028.781704
4,4308.006570,2297.907544,1900.710811,1529.582737,1409.769174,2428.524855,914.355425,910.585742,3347.137973,1438.949915,...,4509.552008,2813.906570,2693.420011,1773.026026,2308.619944,1481.928447,1734.412694,1791.021292,900.699619,1022.437953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,4664.308298,1616.728091,2114.739811,2807.630055,2228.895518,4074.290923,1337.664813,1395.675963,6512.709285,1844.602440,...,18809.202948,6550.251725,5455.545168,3588.860559,4024.846627,2064.479404,3349.121555,2768.854792,615.590971,1396.549045
2430,4657.747965,1611.947581,2128.231508,2800.387667,2216.968521,4043.017804,1348.412941,1390.694090,6578.624767,1843.877192,...,18875.716871,6591.916204,5443.688545,3563.365017,4056.873845,2070.147291,3355.140187,2778.690593,601.983384,1402.715316
2431,4715.034035,1617.483850,2148.326773,2787.237060,2230.617452,4064.806981,1356.902483,1396.951106,6647.440266,1849.346609,...,19060.459302,6630.577198,5434.213963,3584.610698,4093.882893,2076.696490,3373.660143,2816.767013,606.107439,1409.553025
2432,4784.037808,1624.343679,2168.709839,2783.992031,2259.897632,4122.290280,1360.870808,1413.576390,6710.316457,1857.264080,...,19289.785476,6651.937415,5448.373989,3641.899947,4125.348310,2082.678167,3404.968282,2854.137611,618.979886,1417.275944


In [6]:
# 计算每一列的收益率(除了date列)
returns = data.drop('date', axis=1).pct_change()

# # 将date列设为索引
returns.insert(0, 'date', data['date'])
returns.set_index('date', inplace=True)

# 创建一个新的DataFrame来存储标签
labels = returns.copy()

# 设置标签
labels[labels > 0] = 1
labels[labels < 0] = -1
labels[labels == 0] = 0

In [7]:
returns

,美容护理,环保,石油石化,煤炭,通信,计算机,国防军工,机械设备,电力设备,建筑装饰,...,食品饮料,家用电器,汽车,电子,有色金属,钢铁,基础化工,农林牧渔,传媒,非银金融
date,,,,,,,,,,,,,,,,,,,,,
2014/1/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/1/3,-0.003414,-0.003590,-0.006420,-0.015282,-0.000802,0.010625,-0.015067,-0.004675,0.000631,-0.018878,...,-0.009423,-0.023127,-0.013603,0.005773,0.000154,0.000884,-0.009033,-0.007328,0.015972,-0.025125
2014/1/6,-0.026209,-0.017556,-0.012275,-0.039796,-0.029087,-0.026832,-0.039550,-0.030412,-0.024515,-0.033870,...,-0.024886,-0.025639,-0.027692,-0.021779,-0.021762,-0.028710,-0.028604,-0.041350,-0.021641,-0.006281
2014/1/7,0.010191,0.008595,0.004777,-0.005034,0.005231,0.006948,0.006450,0.009019,0.022283,-0.002336,...,0.004791,0.002019,-0.000414,0.015435,0.000559,-0.005864,0.004469,0.000369,0.015224,-0.005728
2014/1/8,-0.015945,0.011179,-0.001652,-0.016649,0.012299,0.019568,0.015293,0.003476,0.006334,-0.006490,...,0.006732,0.006224,-0.000180,0.015792,-0.009819,-0.009995,-0.002358,0.000038,0.041524,0.007590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023/12/25,0.002666,-0.009637,0.003843,0.008846,0.000795,-0.001125,0.013249,0.004329,0.004012,-0.008453,...,0.005661,0.000785,0.010165,0.001275,0.005110,-0.000579,-0.002126,-0.000888,-0.019193,-0.005704
2023/12/26,-0.011544,0.000392,0.001633,-0.014786,-0.018402,-0.022019,-0.013312,-0.016208,-0.007845,-0.008079,...,-0.005272,-0.008192,-0.013671,-0.021006,-0.003778,-0.004885,-0.002551,-0.005874,-0.016090,-0.008255
2023/12/27,-0.006808,0.000671,0.015400,0.009141,0.012159,0.009558,0.000733,0.007836,-0.004431,0.007975,...,0.002738,0.010789,-0.001782,0.016742,0.007002,0.001416,0.000314,0.024800,-0.009101,0.004230


In [8]:
labels

,美容护理,环保,石油石化,煤炭,通信,计算机,国防军工,机械设备,电力设备,建筑装饰,...,食品饮料,家用电器,汽车,电子,有色金属,钢铁,基础化工,农林牧渔,传媒,非银金融
date,,,,,,,,,,,,,,,,,,,,,
2014/1/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/1/3,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
2014/1/6,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2014/1/7,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,...,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
2014/1/8,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,...,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023/12/25,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,...,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2023/12/26,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2023/12/27,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,...,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0


In [9]:
labels = labels[['电力设备',	'基础化工',	'食品饮料',	'环保',	'纺织服饰',	'交通运输',	'商贸零售',	'建筑装饰',	'纺织服饰',	'社会服务']]

In [10]:
labels

,电力设备,基础化工,食品饮料,环保,纺织服饰,交通运输,商贸零售,建筑装饰,纺织服饰,社会服务
date,,,,,,,,,,
2014/1/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014/1/3,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2014/1/6,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2014/1/7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0
2014/1/8,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
2023/12/25,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2023/12/26,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2023/12/27,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0


In [11]:
returns
returns.shape

(2434, 31)

In [12]:
labels = labels.dropna()

In [13]:
# 将列名改为数值
labels.columns = range(labels.shape[1])

In [14]:
labels.head()

,0,1,2,3,4,5,6,7,8,9
date,,,,,,,,,,
2014/1/3,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2014/1/6,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2014/1/7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0
2014/1/8,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2014/1/9,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [15]:
labels.shape

(2433, 10)

In [16]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

In [17]:
# 将labels数据转换为0/1列表
labels_binary = labels.applymap(lambda x: 1 if x == 1 else 0)

In [18]:
labels_binary

,0,1,2,3,4,5,6,7,8,9
date,,,,,,,,,,
2014/1/3,1,0,0,0,0,0,0,0,0,0
2014/1/6,0,0,0,0,0,0,0,0,0,0
2014/1/7,1,1,1,1,1,1,1,0,1,1
2014/1/8,1,0,1,1,0,0,0,0,0,0
2014/1/9,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2023/12/25,1,0,1,0,0,0,0,0,0,0
2023/12/26,0,0,0,1,0,0,0,0,0,0
2023/12/27,0,1,1,1,1,1,0,1,1,1


In [19]:
# 初始化ans_data DataFrame
ans_data = pd.DataFrame(columns=['基准行业', '滞后行业', '支持度', '置信度', '说明'])

In [20]:
# 遍历所有行业组合
for i in range(len(labels_binary.columns)):
    for j in range(i+1, len(labels_binary.columns)):
        base_industry = labels_binary.columns[i]
        lag_industry = labels_binary.columns[j]
        
        # 共涨假设
        base_up = labels_binary[base_industry][:-1] == 1
        lag_up = labels_binary[lag_industry][1:] == 1
        both_up = base_up & lag_up
        support = both_up.mean()
        confidence = both_up.sum() / (labels_binary[base_industry][:-1] == 1).sum()
        if support >= 0.4 and confidence >= 0.8:
            ans_data = ans_data.append({'基准行业': base_industry, '滞后行业': lag_industry, '支持度': support, '置信度': confidence, '说明': '共涨'}, ignore_index=True)
        
        # 共跌假设
        base_down = labels_binary[base_industry][:-1] == 0
        lag_down = labels_binary[lag_industry][1:] == 0
        both_down = base_down & lag_down
        support = both_down.mean()
        confidence = both_down.sum() / (labels_binary[base_industry][:-1] == 0).sum()
        if support >= 0.4 and confidence >= 0.8:
            ans_data = ans_data.append({'基准行业': base_industry, '滞后行业': lag_industry, '支持度': support, '置信度': confidence, '说明': '共跌'}, ignore_index=True)

print(ans_data)

  基准行业 滞后行业       支持度       置信度  说明
0    0    1  0.452528  0.861502  共涨
1    1    4  0.438964  0.805430  共涨
2    1    8  0.438964  0.805430  共涨
3    3    4  0.423757  0.828778  共涨
4    3    8  0.423757  0.828778  共涨
5    4    6  0.433621  0.815301  共涨
6    4    8  0.531854  1.000000  共涨
7    4    8  0.467324  0.999121  共跌
8    6    8  0.433621  0.841979  共涨


In [43]:
import pandas as pd

# 读取交易数据
trade_data = pd.read_csv('电力-化工.csv', encoding='gbk', parse_dates=['date'])

# 初始化参数
initial_capital = 10000000  # 初始资金1000万元
commission_rate_buy = 0.0003  # 买入佣金万分之三
commission_rate_sell = 0.0004  # 卖出佣金万分之三加千分之一印花税
min_commission = 5  # 每笔交易佣金最低5元
slippage = 0.02  # 固定滑点0.02元
risk_free_rate = 0.04  # 无风险利率
benchmark_return = -0.2006  # 基准收益率

# 初始化交易次数和交易记录列表
num_trades = 0
trade_records = []
current_holdings = 0  # 当前持有股票数
current_assets = initial_capital  # 当前资产
last_buy_price = None  # 上次购买价格


# 遍历交易数据执行交易策略
for index, row in trade_data.iterrows():
    date = row['date']
    base_return = row['电力设备']
    lag_return = row['基础化工']
    base_price = row['电力设备-价格']
    lag_price = row['基础化工-价格']
    
    # 检查买入信号
    if base_return >= 0.015 and current_holdings == 0:
        buy_price = lag_price + slippage
        buy_shares = int(current_assets / buy_price)
        buy_commission = max(buy_shares * buy_price * commission_rate_buy, min_commission)
        current_assets -= buy_shares * buy_price * (1 + commission_rate_buy)
        current_holdings += buy_shares
        last_buy_price = buy_price
        num_trades += 1
        trade_records.append([date, buy_price, buy_shares, None, None, None])
        
    # 检查卖出信号
    elif base_return <= -0.015 and current_holdings > 0:
        sell_price = lag_price - slippage
        sell_shares = current_holdings
        sell_commission = max(sell_shares * sell_price * commission_rate_sell, min_commission)
        current_assets += sell_shares * sell_price * (1 - commission_rate_sell)
        current_holdings = 0
        last_buy_price = None
        num_trades += 1
        trade_record = trade_records[-1]
        trade_record[3] = date
        trade_record[4] = sell_price
        trade_record[5] = sell_shares * sell_price - buy_shares * trade_record[1] - buy_commission - sell_commission
        
# 计算策略表现
trade_records_df = pd.DataFrame(trade_records, columns=['买入日期', '买入单价/¥', '买入数量', '卖出日期', '卖出单价/¥', '收益/¥'])

strategy_return = (current_assets - initial_capital) / initial_capital
alpha = strategy_return - risk_free_rate  # 计算阿尔法值
sharpe_ratio = (strategy_return - risk_free_rate) / trade_data['基础化工'].pct_change().std()  # 计算夏普比率
max_drawdown = (trade_records_df['收益/¥'].cumsum() / initial_capital).min() * -1  # 最大回撤率

performance_df = pd.DataFrame({'策略收益率': [strategy_return],
                                '阿尔法值': [alpha],
                                '夏普比率': [sharpe_ratio],
                                '最大回撤率': [max_drawdown]})

In [49]:
print(trade_records_df)

          买入日期   买入单价/¥      买入数量       卖出日期   卖出单价/¥          收益/¥
0   2014-01-07  1752.02      5707 2014-01-09  1755.66  1.376603e+04
1   2014-01-14  1685.27      5941 2014-02-20  1887.61  1.194613e+06
2   2014-02-26  1781.68      6290 2014-03-10  1839.23  3.540000e+05
3   2014-03-17  1797.95      6430 2014-03-20  1839.82  2.610238e+05
4   2014-03-21  1804.07      6553 2014-03-27  1847.23  2.744389e+05
..         ...      ...       ...        ...      ...           ...
178 2023-08-29  3587.62   9199108 2023-09-07  3749.67  1.467017e+09
179 2023-09-22  3594.32   9590109 2023-10-13  3601.30  4.278324e+07
180 2023-10-27  3445.70  10016166 2023-11-02  3550.51  1.025216e+09
181 2023-11-03  3516.18  10106968 2023-11-16  3605.04  8.728694e+08
182 2023-12-21  3337.83  10908521        NaT      NaN           NaN

[183 rows x 6 columns]


In [47]:
strategy_return = (current_assets + 3337.83*10908521    - initial_capital) / initial_capital
alpha = strategy_return - risk_free_rate  # 计算阿尔法值
sharpe_ratio = (strategy_return - risk_free_rate) / trade_data['基础化工'].pct_change().std()  # 计算夏普比率
max_drawdown = (trade_records_df['收益/¥'].cumsum() / initial_capital).min() * -1  # 最大回撤率

performance_df = pd.DataFrame({'策略收益率': [strategy_return],
                                '阿尔法值': [alpha],
                                '夏普比率': [sharpe_ratio],
                                '最大回撤率': [max_drawdown]})

In [48]:
print(performance_df)

         策略收益率         阿尔法值        夏普比率     最大回撤率
0  3638.986789  3638.946789  179.508887 -0.001377
